<center> </center>

<center><font size=5 face="Helvetica" color=#EE4B2B><b>
Pytorch Tutorial: Automatic Differentiation with torch.autograd
</b></font></center>

<center><font face="Helvetica" size=3><b>Ang Chen</b></font></center>
<center><font face="Helvetica" size=3>July, 2024</font></center>

***

When training neural networks, the most frequently used algorithm is **back propagation**.
In this algorithm, parameters (model weights) are adjusted according to the **gradient** of the loss
function with respect to the given parameter.

To compute those gradients, PyTorch has a built-in differentiation engine called $\texttt{torch.autograd}$.
It supports automatic computation of gradient for any computational graph.

Consider the simplest one-layer neural network, with input $\texttt{x}$, parameters $\texttt{w}$ and $\texttt{b}$, and some loss function.
It can be defined in PyTorch in the following manner:

In [1]:
import torch

x = torch.ones(5) # Input tensor
y = torch.zeros(3) # Expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b

print('x:', x)
print('y:', y)
print('w:', w)
print('b:', b)
print('z:', z)

x: tensor([1., 1., 1., 1., 1.])
y: tensor([0., 0., 0.])
w: tensor([[ 0.7920, -0.2027, -1.1139],
        [-0.3537, -0.5665, -0.3595],
        [-0.3884, -0.8602,  0.1977],
        [-0.3287, -0.4994,  0.6999],
        [ 2.5346,  0.5535,  1.4998]], requires_grad=True)
b: tensor([-1.1476, -0.7146,  1.4001], requires_grad=True)
z: tensor([ 1.1081, -2.2899,  2.3242], grad_fn=<AddBackward0>)


In [2]:
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
print('loss:', loss)

loss: tensor(1.3025, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


# Tensors, Functions and Computational Graphs

This code defines the following **computational graph**:

<img src="./figures/computational_graph.png" alt="Computational graph" width="800">

In this work, $w$ and $b$ are **parameters**, which we need to optimize.
Thus, we need to be able to compute the gradients of loss function with respect to those variables.
In order to do that, we set the $\texttt{requires\_grad}$ property of those tensors.

A function that we apply to tensors to construct computational graph is in fact an object of class $\texttt{Function}$.
This object knows how to compute the function in forward direction, and also how to compute its derivative during the *backward propagation* step.
A reference to the backward propagation is stored in $\texttt{grad\_fn}$ property of a tensor.
You can find more information of $\texttt{Function}$ in the documentation.

In [3]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x10a9078e0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x10a907190>


# Computing Gradients

To optimize weights of parameters in the neural network, we need to compute the derivatives of our loss function with respect to parameters, namely, we need $\frac{\partial\text{loss}}{\partial w}$ and $\frac{\partial\text{loss}}{\partial b}$ under some fixed values of $x$ and $y$.
To compute those derivatives, we call $\texttt{loss.backward()}$, and then retrieve the values from $\texttt{w.grad}$ and $\texttt{b.grad}$:

In [4]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2506, 0.0307, 0.3036],
        [0.2506, 0.0307, 0.3036],
        [0.2506, 0.0307, 0.3036],
        [0.2506, 0.0307, 0.3036],
        [0.2506, 0.0307, 0.3036]])
tensor([0.2506, 0.0307, 0.3036])


# Disabling Gradient Tracking

By default, all tensors with $\texttt{requires\_grad=True}$ are tracking their computational history and support gradient computation.
However, there are some cases when we do not need to do that, for example, when we have trained the model and just want to apply it to some input data, i.e., we only want to do *forward* computatoins through the network.
We can stop tracking computations by surrounding our computation code with $\texttt{torch.no\_grad()}$ block:

In [5]:
z = torch.matmul(x, w) + b
z.requires_grad

True

In [6]:
with torch.no_grad():
    z = torch.matmul(x, w) + b
z.requires_grad

False

Another way to achieve the same result is to use the $\texttt{detach()}$ method on the tensor:

In [7]:
z = torch.matmul(x, w) + b
print(z.requires_grad)
z_det = z.detach()
z_det.requires_grad

True


False

There are reasons you might want to disable gradient tracking:
 * To mark some parameters in your neural network as **frozen parameters**.
 * To **speed up computations** when you are only doing forward pass, because computations on tensors that do not track gradients would be more efficient.

# More on Computational Graphs

Conceptually, autograd keeps a record of data (tensors) and all executed operations (along with the resulting new tensors) in a directed acyclic graph (DAG) consisting of Function objects.
In this DAG, leaves are the input tensors, roots are the output tensors.
By tracing this graph from roots to leaves, you can automatically compute the gradients using the chain rule.

In a forward pass, autograd does two things simultaneously:
 * run the requested operation to compute a resulting tensor